### Problem 3

#### a) 
<font size = 3 color = black face = "Times New Roman"> In this problem, we're required to solve the following linear system with the SOR method.
$$
\begin{cases}
3x_1 - x_2+ x_3 = -1\\
-x_1 +3x_2 -x_3 = 7\\
x_1 - x_2 +3x_3 = -7\\
\end{cases}
$$

<font size = 3 color = black face = "Times New Roman"> First of all, let's rewrite the linear system in the matrix form and input related matrices. define 
$
A = \begin{pmatrix} 
3 & -1 & 1\\
-1 & 3 & -1\\
1 & -1 & 3\\
\end{pmatrix}
$, $
b = \begin{pmatrix} 
-1\\
7\\
-7\\
\end{pmatrix}
$
, then the system can be written as
$$
Ax = 
\begin{pmatrix} 
3 & -1 & 1\\
-1 & 3 & -1\\
1 & -1 & 3\\
\end{pmatrix}
\begin{pmatrix} 
x_1\\
x_2\\
x_3\\
\end{pmatrix}
= 
\begin{pmatrix} 
-1\\
7\\
-7\\
\end{pmatrix}
= b
$$

In [130]:
import numpy as np # Input matrices A and b
A = np.array([[3, -1, 1],
              [-1, 3, -1],
              [1, -1, 3]])
b = np.array([-1, 7, -7])

<font size = 3 color = black face = "Times New Roman"> After that, we define a function to use the SOR method to solve linear systems.

In [133]:
eps = 1e-10 # Set the accuracy (epsilon).

In [131]:
from numpy import array, zeros, zeros_like, dot, allclose
def SOR(A, b, omega, N=25, x=None):
# omega is the relaxation parameter,N is the maximum iteration number, x is the initial guess, eps is the accuracy.
    if x is None:
        x = zeros_like(b)
    for it_count in range(N): 
        x_new = zeros_like(x)
        print("Iteration {0}: {1}".format(it_count, x))
        for i in range(A.shape[0]):
            s1 = dot(A[i, :i], x_new[:i])
            s2 = dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (1 - omega) * x[i] + omega * (b[i] - s1 - s2) / A[i, i] # SOR interation
        if allclose(x, x_new, rtol=1e-10):
            break
        x = x_new
    return x

In [132]:
SOR(A, b, omega = 1.1, N =25, x= [2, 2, 2])

Iteration 0: [2, 2, 2]
Iteration 1: [ 0  3 -1]
Iteration 2: [ 1  2 -2]


array([ 1,  2, -2])

<font size = 3 color = black face = "Times New Roman">Finally, we get the solution 
$$
\begin{cases}
x_1 = 1\\
x_2 = 2\\
x_3 = -2\\
\end{cases}
$$
#### b)
<font size = 3 color = black face = "Times New Roman">Empirically, we guess the optimal $\omega = $
Then, let's use the formula below to calculate the theoretical prediction of the optimal relaxation parameter $\omega$
$$
\omega_\text{opt} = 1+\left(\frac{\lambda_J}{1+\sqrt{1-\lambda_J^2}}\right)^2 
$$
$\lambda_J$ is the spectral radius calculated  in the Jacobi method.  
Therefore, we should find $\lambda_J$ first, and here we introduce the power method.

In [133]:
def power(A, x0, epsilon = 1e-10):
    i = 0
    x1 = x0
    a0 = 0
    a1 = 1
    while abs(a1 - a0) > epsilon:
        x0 = x1
        x1 = np.dot(A, x0)
        a0 = a1
        a1 = np.dot(x0, x1)/np.dot(x0,x0)
        x1 = normalise(x1)
        i = i + 1
    return x1 ,i, a1
x = normalise(np.array([1, 1, 1]))
power(A, x, epsilon = 1e-10)

(array([ 0.57735047, -0.57734987,  0.57735047]), 17, 4.999999999995573)

<font size = 3 color = black face = "Times New Roman"> Now we have $\lambda_J \approx 0.57746778$

In [4]:
lamda_j = 5
omega_op =1 + (lamda_j / (1 + (1 - lamda_j  ** 2)**0.5))**2
omega_op

(0.07999999999999985-0.3919183588453087j)

<font size = 3 color = black face = "Times New Roman"> Finally, we have $\omega_\text{opt} \approx 1.101000671653077$

In [22]:
from pprint import pprint


def gauss_seidel(A, b, N=25, x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed                                                                                                                                                            
    if x is None:
        x = zeros_like(b)

    # Iterate for N times  
    for it_count in range(N):
        x_new = zeros_like(x)
        print("Iteration {0}: {1}".format(it_count, x))
        for i in range(A.shape[0]):
            s1 = dot(A[i, :i], x_new[:i])
            s2 = dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if allclose(x, x_new, rtol=1e-8):
            break
            
            
        x = x_new
    return x

In [179]:
gauss_seidel(A, b, N=25, x= [-5,8,10])

Iteration 0: [-5, 8, 10]
Iteration 1: [-1  5  0]
Iteration 2: [ 1  2 -2]


array([ 1,  2, -2])

### Problem 4

<font size = 3 color = black face = "Times New Roman"> Once again, we're going to solve the Problem $Ax = b$, but using the conjugate gradient method. In this problem $A$ is shown below and $b$ is generated by the normal distributed random numbers. 
$$
A = \begin{pmatrix}
2 &-1  &  & &\\
-1 & 2 &-1  & \\
& -1 & 2 &-1  \\
& & -1  &2 &-1 \\
& & & -1 & 2
\end{pmatrix}
$$

<font size = 3 color = black face = "Times New Roman"> First of all, let's input $A$ and generate $b$. Sinc  $A$ is already a positive symmetric matrix, we don't need to addtional operations.

In [137]:
def matrix_A(N):
    A = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i==j:
                A[i][j] = 2
            elif i == j - 1 or i == j + 1:
                A[i][j] = -1
    return A
A = matrix_a(5) # Create the matrix A
b = np.random.normal(size = (5)) # Create b with the normal distributed random numbers.

<font size = 3 color = black face = "Times New Roman"> Then we define a function to solve problems with conjugate gradient method.

In [135]:
def conjugate_grad(A, b, x=None):
    n = len(b)
    if not x:
        x = np.ones(n)
    r = b - np.dot(A, x)
    p = r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Ap = np.dot(A, p)
        alpha = r_k_norm / np.dot(p, Ap)
        x += alpha * p
        r -= alpha * Ap
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-5:
            print('Itr:{}'.format(i))
            break
        p = r + beta * p
    return x

In [136]:
conjugate_grad(A, b, x=None)

Itr:9


array([ 0.85010704,  0.60079985,  0.53916831, -0.2716156 , -0.57425564])

### Problem 6

<font size = 3 color = black face = "Times New Roman"> Firstly, enter the matrix we're going to deal with

In [124]:
A = np.array([[4, -1j, 2], [1j, 2, 2 + 7j], [2, 2 - 7j, -2]])

In [127]:
import numpy as np
def minimum_eigen_finder(A, N = 25, x = None):
    if not x:
        x = normalise(np.array([1, 1, 1]))
    else:
        x = normalise(np.array(x)) # Step1: Choose a normalised vector x0   
    for i in range(N):    
        x_new = np.linalg.solve(A,x) # Step2: Get the next x
        x_new = normalise(x_new)
        if np.linalg.norm(x_new - x, ord=2) < 10**(-8):
            break
        x = x_new
    return x

def rayleigh_quotient(A,x):
    return np.dot(x, np.dot(A, x))

In [129]:
solution = minimum_eigen_finder(A, N = 25, x = None)
rayleigh_quotient(A, solution)

(3.1896027772323547-2.9426682307850324e-09j)